In [1]:
import sys
sys.path.append('../../')

# BERT_FINE_TUNE_PATH = '../../../bert_fine_tune/'
# sys.path.append(BERT_FINE_TUNE_PATH)

In [2]:
# from textpair.preprocess.dummy_preprocessor import DummyPreprocessor
# from textpair.analyze.bert_analyzer import BertAnalyzer
# from textpair.vectorize.bert_vectorizer import BertVectorizer
# from textpair.model.bert_model import BertModel
# from textpair.pair_ann import PairAnn
# from textpair.semantic.base_semantic import BaseSemantic

import logging

logging.basicConfig(level=logging.INFO)

import torch
from pytorch_pretrained_bert.modeling_fine_tune import BertForPointWiseClassification
from pytorch_pretrained_bert import BertTokenizer

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_pretrained_bert.modeling_fine_tune:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
FINE_TUNED_PATH = '/efs/fine_tune/lcqmc/pointwise_old/lcqmc_fine_tune_40_1_1e-5/'

In [4]:
model = BertForPointWiseClassification.from_pretrained(FINE_TUNED_PATH)
tokenizer = BertTokenizer.from_pretrained(FINE_TUNED_PATH)

INFO:pytorch_pretrained_bert.modeling_fine_tune:loading weights file /efs/fine_tune/lcqmc/pointwise_old/lcqmc_fine_tune_40_1_1e-5/pytorch_model.bin
INFO:pytorch_pretrained_bert.modeling_fine_tune:loading configuration file /efs/fine_tune/lcqmc/pointwise_old/lcqmc_fine_tune_40_1_1e-5/config.json
INFO:pytorch_pretrained_bert.modeling_fine_tune:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file /efs/fine_tune/lcqmc/pointwise_old/lcqmc_fine

In [5]:
def bert_sim(text1, text2):
    tokens1 = ['[CLS]'] + tokenizer.tokenize(text1) + ['[SEP]']
    tokens2 = ['[CLS]'] + tokenizer.tokenize(text2) + ['[SEP]']
    ids1 = tokenizer.convert_tokens_to_ids(tokens1)
    ids2 = tokenizer.convert_tokens_to_ids(tokens2)
    segs1 = [0] * len(ids1)
    segs2 = [0] * len(ids2)
    tokens_tensor1 = torch.tensor([ids1])
    segments_tensor1 = torch.tensor([segs1])
    tokens_tensor2 = torch.tensor([ids2])
    segments_tensor2 = torch.tensor([segs2])
    model.eval()
    with torch.no_grad():
        logits, vec1, vec2 = model(tokens_tensor1, tokens_tensor2, segments_tensor1, segments_tensor2)
        probs = torch.softmax(logits, dim = -1)
    return probs[0, 1].item(), vec1, vec2

# 一般测试

In [6]:
text1 = "我很高兴"
text2 = "我很开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.961258053779602

In [7]:
text1 = "我很开心"
text2 = "我很高兴"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.961258053779602

In [8]:
text1 = "我很高兴"
text2 = "我特别特别开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8213762640953064

In [9]:
text1 = "我很高兴"
text2 = "我其实觉得自己很开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7699229717254639

In [10]:
text1 = "我特别特别开心"
text2 = "我其实觉得自己很开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9371229410171509

In [11]:
text1 = "我很高兴"
text2 = "我不开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.0798477828502655

In [12]:
text1 = "我很高兴"
text2 = "我不高兴"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.06977882981300354

In [13]:
text1 = "我很高兴"
text2 = "我很高兴"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9942808151245117

In [14]:
text1 = "我很伤心"
text2 = "我很难过"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9801499843597412

In [15]:
text1 = "真好"
text2 = "不错"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9436234831809998

In [16]:
text1 = "高兴"
text2 = "开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8590299487113953

In [17]:
text1 = "大家好才是真的好"
text2 = "大家好才是真的好"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9949809908866882

In [18]:
text1 = "为什么能开出腾讯信用却没有微粒贷朋友的没用腾讯信用却有30000的额度呢"
text2 = "我钱包里没有你们这个应用"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.0014409964205697179

In [19]:
text1 = "我也不知道"
text2 = "好吧"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.13305222988128662

In [20]:
text1 = "深度学习"
text2 = "机器学习"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7389079332351685

In [21]:
text1 = "机器学习"
text2 = "深度学习"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7389079332351685

In [22]:
text1 = "人民团体是什么"
text2 = "人民团体是指"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.980027973651886

# 百度-车牌，不太一致

In [23]:
text1 = "车头如何放置车牌"
text2 = "前牌照怎么装"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7907896637916565

In [24]:
text1 = "车头如何放置车牌"
text2 = "如何办理北京车牌"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.800718367099762

In [25]:
text1 = "车头如何放置车牌"
text2 = "后牌照怎么装"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.531283974647522

# 百度-信号

In [26]:
text1 = "信号忽强忽弱"
text2 = "信号忽高忽低"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6957070231437683

In [27]:
text1 = "信号忽强忽弱"
text2 = "信号忽左忽右"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.15018492937088013

In [28]:
text1 = "信号忽强忽弱"
text2 = "信号忽然中断"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.4576272964477539

# 百度-机器学习

In [29]:
text1 = "如何学好深度学习"
text2 = "深入学习习近平讲话材料"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.05825630575418472

In [30]:
text1 = "如何学好深度学习"
text2 = "机器学习教程"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6349290013313293

In [31]:
text1 = "如何学好深度学习"
text2 = "人工智能教程"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.4203011095523834

# 百度-香蕉的翻译，偏小但排序一致

In [32]:
text1 = "香蕉的翻译"
text2 = "香蕉用英文怎么说"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7851580381393433

In [33]:
text1 = "香蕉的翻译"
text2 = "香蕉怎么吃"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.01105554960668087

In [34]:
text1 = "香蕉的翻译"
text2 = "桔子用英文怎么说"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.4990476965904236

# 百度-腹泻，排序有差别

In [35]:
text1 = "小儿腹泻偏方"
text2 = "宝宝拉肚子偏方"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.903954267501831

In [36]:
text1 = "小儿腹泻偏方"
text2 = "小儿感冒偏方"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9099643230438232

In [37]:
text1 = "小儿腹泻偏方"
text2 = "腹泻偏方"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9675581455230713

# 百度-LOL，数值偏小，但排序一致

In [38]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "lol攻略"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.18875348567962646

In [39]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "英雄联盟服务器升级"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6169710159301758

In [40]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "怎么打好lol"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.30981332063674927

# 百度-红米

In [41]:
text1 = "红米更新出错"
text2 = "红米升级系统出错"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8867897391319275

In [42]:
text1 = "红米更新出错"
text2 = "红米账户出错"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8085059523582458

In [43]:
text1 = "红米更新出错"
text2 = "如何买到小米手机"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.004510632250458002

# 百度-李彦宏

In [44]:
text1 = "李彦宏是百度公司创始人"
text2 = "百度是李彦宏创办的"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9763168096542358

In [45]:
text1 = "李彦宏是百度公司创始人"
text2 = "马化腾创办了腾讯公司"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7467296123504639

In [46]:
text1 = "李彦宏是百度公司创始人"
text2 = "姚明是NBA的著名球星"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.20833785831928253

# 百度-中国历史

In [47]:
text1 = "中国有五千年的历史"
text2 = "中国是个历史悠久的国家"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9515589475631714

In [48]:
text1 = "中国有五千年的历史"
text2 = "中国有很多少数民族"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.3454322814941406

In [49]:
text1 = "中国有五千年的历史"
text2 = "中国有13亿人口"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.703502893447876

# 百度-北京奥运会，偏小，但数值一致

In [50]:
text1 = "北京成功申办了2008年奥运会"
text2 = "2008年奥运会在北京举行"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6741499900817871

In [51]:
text1 = "北京成功申办了2008年奥运会"
text2 = "伦敦奥运会在2012年举行"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.27273795008659363

In [52]:
text1 = "北京成功申办了2008年奥运会"
text2 = "东京奥运会即将举办"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.31581932306289673